In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import time
import torch
import numpy as np
import pandas as pd
from PIL import Image
from moviepy.editor import *
from torchvision import transforms

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3645440/45929032 bytes (7.9%)7774208/45929032 bytes (16.9%)11894784/45929032 bytes (25.9%)15974400/45929032 bytes (34.8%)20054016/45929032 bytes (43.7%)24043520/45929032 bytes (52.3%)28090368/45929032 bytes (61.2%)32227328/45929032 bytes (70.2%)36208640/45929032 bytes (78.8%)40255488/45929032 bytes (87.6%)44457984/45929032 bytes (96.8%)45929032/45929032 bytes (100.0%)
  Done
File saved as /root

In [ ]:
import pandas as pd
df = pd.read_csv('drive/MyDrive/it416/data.csv')
df

,VideoID,Description
0,-4wsuPCjDBc_5_15,a squirrel is eating a peanut in it s shell
1,-4wsuPCjDBc_5_15,a chipmunk is eating
2,-4wsuPCjDBc_5_15,a chipmunk is eating a peanut
3,-4wsuPCjDBc_5_15,a chipmunk is eating a nut
4,-4wsuPCjDBc_5_15,a squirrel is eating a nut
...,...,...
80783,zxB4dFJhHR8_1_9,a girl riding a bicycle
80784,zxB4dFJhHR8_1_9,a smiling girl wearing backpack is riding a bike
80785,zxB4dFJhHR8_1_9,the girl rode her bike at the beach
80786,zzit5b_-ukg_5_20,a boy is doing exercise by cycle


In [ ]:
# Note: Changed to last 800 from random 800
videoIDs = list(df.VideoID.unique())
videoIDs=videoIDs[1170:]
print(len(videoIDs))

800


In [ ]:
def extract_frames_equally_spaced(frames, K):
        n_frames = len(frames)
        splits = np.array_split(range(n_frames), K)
        idx_taken = [s[0] for s in splits]
        sub_frames = []

        for idx in idx_taken:
          sub_frames.append(frames[idx])
        return sub_frames

def pad_frames(frames, limit, jpegs=False):
        last_frame = frames[-1]
        if jpegs:
            frames_padded = frames + [last_frame]*(limit-len(frames))
        else:
            padding = np.asarray([last_frame * 0.]*(limit-len(frames)))
            frames_padded = np.concatenate([frames, padding], axis=0)
        return frames_padded

def video_to_frames(input_loc,K):
    cap = cv2.VideoCapture(input_loc)
    
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
    count = 0

    frames=[]
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            continue
        frames.append(frame)
        count = count + 1
        
        if (count > (video_length-1)):
            cap.release()
            break
    if len(frames) < K:
        frames = pad_frames(frames,K)
    else:
        frames = extract_frames_equally_spaced(frames,K)
    
    return frames

def resize_frames(frames):
  new_frames = []
  for frame in frames:
    new_frame = cv2.resize(frame,(224,224))
    new_frames.append(new_frame)
  
  return new_frames

K=28
input_data = []

for id in videoIDs:
  path = 'drive/MyDrive/it416/YouTubeClips/'
  path+=id
  path+='.avi'
  print(id)

  frames = video_to_frames(path,K)
  frames = resize_frames(frames)

  input_data.append(frames)

input_data = np.array(input_data)
np.save('input_data.npy',input_data)
print(input_data.shape)

In [ ]:
#frames
!wget 'https://drive.google.com/uc?id=10uilSyuHEbl6f6zJgh89_-uRfev8rQ8N&confirm=t' -O 'input_data_random_800.npy'

In [ ]:
#videoIDs
!wget 'https://drive.google.com/uc?id=1-0ZOCy3t7sO5bowBc4AcmOfLacqcRlEI&confirm=t' -O 'videoIDs_random_800.npy'

In [ ]:
input_data = np.load('input_data_random_800.npy')
videoIDs = np.load('videoIDs_random_800.npy')